# RAG Baseline

In [ ]:
import os
import requests
from dotenv import load_dotenv

from langchain_upstage import UpstageDocumentParseLoader
from langchain_upstage import UpstageEmbeddings
from langchain_upstage import ChatUpstage

from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore
# from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain_teddynote.community.pinecone import create_index

from rag_utils import logging

# API 키 정보 로드
load_dotenv()

In [ ]:
# LangSmith 시작
logging.langsmith("test-RAG")

In [ ]:
# 단계 1: 문서 로드(Load Documents)  # PyMuPDFLoader
file_path = "./data/politics_and_the_law_page_10-38.pdf"
loader = PyMuPDFLoader(file_path)
docs = loader.load()
print(f"문서의 페이지수: {len(docs)}")

In [ ]:
# matadata 확인
docs[0].__dict__

In [ ]:
# 단계 2-1: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
split_documents = text_splitter.split_documents(docs)
print(f"분할된 청크의수: {len(split_documents)}")

In [ ]:
dict(split_documents[0])

In [ ]:
for doc in split_documents:
    print(doc.page_content)
    print("+"*100)

In [ ]:
# 단계 2-2: 문서 전처리
from langchain_teddynote.community.pinecone import preprocess_documents

contents, metadatas = preprocess_documents(
    split_docs=split_documents,
    metadata_keys=["source", "page", "author"],
    min_length=5,
    use_basename=True,
)

In [ ]:
contents[0]

In [ ]:
metadatas.keys()

In [ ]:
metadatas['source'][0]

In [ ]:
# 문서 개수 확인, 소스 개수 확인, 페이지 개수 확인
len(contents), len(metadatas["source"]), len(metadatas["page"])

In [ ]:
# 단계 3: 임베딩(Embedding) 생성
embeddings = UpstageEmbeddings(model="embedding-passage")  # embedding-query 또는 embedding-passage

In [ ]:
# 단계 4: DB 생성(Create DB) 및 저장
# vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

pc = Pinecone(api_key=os.environ['PINECONE_API_KEY'])

index_name = "quickstart"
if index_name not in [index_info["name"] for index_info in pc.list_indexes()]:
    pc.create_index(
        name=index_name,
        dimension=4096, 
        metric="dotproduct",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        ) 
    )
    print(f"{index_name} has been successfully created")
else:
    print(f"{index_name} is already exists.")


vectorstore = PineconeVectorStore.from_documents(split_documents, embeddings, index_name=index_name)

In [ ]:
# 단계 5: 검색기(Retriever) 생성
retriever = vectorstore.as_retriever()

In [ ]:
# 단계 6: 프롬프트 생성(Create Prompt)
prompt = PromptTemplate.from_template(
    """너는 입력을 보고 틀린 부분에 대해서 피드백을 주는 선생님이야.
입력과 관련있는 정보를 참고해서 피드백을 생성해줘.
참고한 정보의 페이지도 같이 알려줘.
만약 틀린 부분이 없을 경우, 칭찬 한문장 작성해줘.

#정보: 
{context}

#입력:
{question}

#답:"""
)

In [ ]:
# 단계 7: 언어모델(LLM) 생성
llm = ChatUpstage(model="solar-pro")

In [ ]:
# 단계 8: 체인(Chain) 생성
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
chain

In [ ]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "정치는 개인과 개인, 개인과 집단, 집단과 집단 등 사회 구성원들 사이에서 발생하는 이해관계의 대립과 충돌을 해결하여 사회의 통합과 질서를 유지하는 행위를 뜻한다."

response = chain.invoke(question)
print(response)

In [ ]:
question = "라스웰은 정치의 본질은 언제 어디서나 상반되는 가치와 감정을 동시에 포함한다는 데 있다 라고 했다"

response = chain.invoke(question)
print(response)

In [ ]:
# 기존에 생성된 벡터 DB를 불러와서 유사성 검색하기

pc = Pinecone()
index = pc.Index(index_name)
vectorstore = PineconeVectorStore(index=index, embedding=embeddings)

question = "라스웰은 정치의 본질은 언제 어디서나 상반되는 가치와 감정을 동시에 포함한다는 데 있다 라고 했다"
results = vectorstore.similarity_search(question, k=2)
results